In [2]:
import re
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

file_path = "/home/pan251/accel-sim-framework-pim/pip.txt"
done_cycles = {}
start_cycles = {}

color_array = px.colors.qualitative.Plotly

freq = 0
last_done = 0

xbar_to_layer = {}


with open(file_path, "r") as file:
    for line in file:
        match = re.match(r"xbar (\d*) done layer, (\d*)", line)
        if match:
            xbar_id = int(match.group(1))
            cycle = int(match.group(2))
            if xbar_id not in done_cycles:
                done_cycles[xbar_id] = []
            done_cycles[xbar_id].append(cycle)
            last_done = cycle

        match = re.match(r"xbar (\d*) start compute, (\d*)", line)
        if match:
            xbar_id = int(match.group(1))
            cycle = int(match.group(2))
            if xbar_id not in start_cycles:
                start_cycles[xbar_id] = []
            start_cycles[xbar_id].append(cycle)

        match = re.match(r"-gpgpu_clock_domains (\d*.\d*):.*", line)
        if match:
            freq = float(match.group(1)) * 1e6

        match = re.match(r"layer (\d*) mappped to xbar (\d*)", line)
        if match:
            layer = match.group(1)
            xbar = int(match.group(2))
            xbar_to_layer[xbar] = "layer " + layer


In [13]:

to_plot = pd.DataFrame(columns=["layer", "xbar", "start", "done", "duration", "color", "text"])
for xbar, starts in start_cycles.items():
    if (len(starts) != len(done_cycles[xbar])):
        done_cycles[xbar].append(last_done)
    assert(len(starts) == len(done_cycles[xbar]))
    dones = done_cycles[xbar]
    for index, start in enumerate(starts):
        start = start / freq * 1e3
        done = dones[index] / freq * 1e3
        duration = (done - start)
        # format
        # duration = "{:.5f}".format(duration)
        print(duration)
        
        to_plot = pd.concat([to_plot, pd.DataFrame({
            "layer": [xbar_to_layer[xbar]],
            "xbar": [str(xbar)], 
            "start": [start], 
            "duration": [duration], 
            "done": [done], 
            "color": [color_array[index % len(color_array)]],
            "text": [f"{duration:.2f} ms"]
            })])
        
fig = go.Figure()
fig.add_trace(go.Bar(
    x = to_plot["duration"],
    y = [to_plot["layer"],"xbar " + to_plot["xbar"]],
    orientation = "h",
    base = to_plot["start"],
    marker_color = to_plot["color"],
    # add text duration
    text = to_plot["text"],
)
)

# change size
fig.update_layout(
    autosize=False,
    width=1200,
    height=300,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()
# write jpg
fig.write_image("pip.jpg", width=1920, height=1080, scale=2)
fig.write_html("./pip.html")

14.674274734982333
14.674274734982333
14.674274734982333
14.674274734982333
14.674274734982333
7.9778330388692495
1.5956872791519423
1.5956890459363962
1.5956890459363962
1.5956890459363962
1.5956890459363962
4.786424911660777
4.786424028268549
4.786424028268549
4.786424028268549
4.786424028268556
4.786456713780922
4.78645759717314
4.786457597173147
4.786457597173133
4.78645759717314
4.7864284452296815
4.786430212014132
4.786430212014139
4.786430212014132
4.786430212014139
1.5956872791519423
1.5956872791519459
1.5956872791519388
1.59568727915196
1.5956872791519459


In [2]:
import onnx
import json

model = onnx.load("/home/pan251/accel-sim-framework-pim/resnet50.onnx")

# for node in model.graph.node:
#     print(node)
#     print("----------------------")

from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import Parse

json_obj = MessageToJson(model.graph.node[0])

# json to protobuf
json_protobuf = Parse(json_obj, onnx.NodeProto())

model.graph.SerializeToString()

AttributeError: 'google._upb._message.RepeatedCompositeContainer' object has no attribute 'SerializeToString'

In [5]:
import torch

resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=False)

print(resnet50)

Using cache found in /home/pan251/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [27]:
import torchinfo
import torchview
import graphviz
graphviz.set_jupyter_format('png')

# torchinfo.summary(resnet50, input_size=(1, 3, 224, 224), device='cpu',

#     col_names=["input_size", "output_size", "kernel_size", "mult_adds"],
#     depth=9)

model_graph = torchview.draw_graph(model, input_size=(1, 3, 224, 224), device='cpu',
                                   expand_nested=True, depth=9)

model_graph.visual_graph

AttributeError: training